In [1]:
import pandas as pd
import yfinance as yf
import re
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import time
from math import log, sqrt, pi, exp
from scipy.stats import norm

In [2]:
def initializeUnderlyingDataAndDicts():
    global underlyingData, underlyingPriceDict, underlying15DVolDict, underlying30DVolDict, underlying60DVolDict, underlyingYZVolDict

    # Opening underlying file
    underlyingData = pd.read_csv("data/AAPL.csv")
    
    # Prepare all the underlying volatility dicts
    underlyingData["PriceChange"] = underlyingData["Close"] - underlyingData["Close"].shift(-1)
    underlyingData["PercentChange"] = underlyingData["PriceChange"] / (underlyingData["Close"] + underlyingData["PriceChange"])
    underlyingData['15DayVol'] = underlyingData["PercentChange"].rolling(15).std() * (252 ** 0.5)
    underlyingData['30DayVol'] = underlyingData["PercentChange"].rolling(30).std() * (252 ** 0.5)
    underlyingData['60DayVol'] = underlyingData["PercentChange"].rolling(60).std() * (252 ** 0.5)
    
    # Code from here: https://harbourfronts.com/garman-klass-yang-zhang-historical-volatility-calculation-volatility-analysis-python/
    underlyingData['YZVol'] = np.sqrt(252 / 30 * pd.DataFrame.rolling(np.log(underlyingData.loc[:, "Open"] / underlyingData.loc[:, "Close"].shift(1)) ** 2 +
                         0.5 * np.log(underlyingData.loc[:, "High"] / underlyingData.loc[:, "Low"]) ** 2 -
                         (2 * np.log(2) - 1) *
                         np.log(underlyingData.loc[:, "Close"] / underlyingData.loc[:, "Open"]) ** 2,
                         window=30).sum())
    
    # Remove NaN
    underlyingData = underlyingData.dropna()
    underlyingData.reset_index(drop=True)
    
    # Populate global dictionaries
    underlyingPriceDict = dict(zip(underlyingData.Date, underlyingData.Close))
    underlying15DVolDict = dict(zip(underlyingData.Date, underlyingData["15DayVol"]))
    underlying30DVolDict = dict(zip(underlyingData.Date, underlyingData["30DayVol"]))
    underlying60DVolDict = dict(zip(underlyingData.Date, underlyingData["60DayVol"]))
    underlyingYZVolDict = dict(zip(underlyingData.Date, underlyingData.YZVol))



def getObjectiveClosePrice(underlyingData, date):
    # Gets the close price on a date given objective data. None if date DNE
    # Consider using a hash map.
    try:
        return float(underlyingData.loc[underlyingData['Date'] == date]['Close'])
    except:
        return None

def getOptionProfit(expireDate, strike):
    # Gets the profit of an option given its expiration date and strike price
    endPrice = underlyingPriceDict[expireDate]
    if endPrice == None:
        return 0
    return max(endPrice - strike, 0)    
    
def getAndProcessOneYearOptionData():
    initializeUnderlyingDataAndDicts()
    finalDay = '2022-04-27'
    
    # Creating masterDf 
    masterDf = pd.read_csv("data/aapl_eod_202101.txt")
    for i in range(2, 13):
        masterDf = masterDf.append(pd.read_csv("data/aapl_eod_2021" + str(i).zfill(2) + ".txt"))
    
    # Dropping irrelevant columns
    cols_to_drop = [0, 1, 3, 6, 14, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
    masterDf.drop(masterDf.columns[cols_to_drop],axis=1,inplace=True)
    
    # Modifying column names for readability
    col_names = []
    for x in list(masterDf.columns):
        x = x.replace(' [', '').replace(']', '')
        col_names.append(x)

    masterDf.columns = col_names

    # Converting all but a few columns to numeric and removing NaN
    for i in range(len(col_names)):
        if i == 0 or i == 2 or i == 11:
            continue
        masterDf[col_names[i]] = pd.to_numeric(masterDf[col_names[i]], errors='coerce')
        masterDf = masterDf[masterDf[col_names[i]].notna()]
    
    # Stripping strings in problematic columns
    masterDf['QUOTE_DATE'] = masterDf['QUOTE_DATE'].str.strip()
    masterDf['C_SIZE'] = masterDf['C_SIZE'].str.strip()
    masterDf['EXPIRE_DATE'] = masterDf['EXPIRE_DATE'].str.strip()

    # Risk free rate column
    riskFreeRateDf = pd.read_csv("data/RiskFreeRate.csv")
    riskFreeRateDict = dict(zip(riskFreeRateDf.DATE, riskFreeRateDf.DTB3))
    masterDf['RISK_FREE_RATE'] = masterDf['QUOTE_DATE'].map(riskFreeRateDict)
    masterDf['RISK_FREE_RATE'] = pd.to_numeric(masterDf['RISK_FREE_RATE'], errors='coerce')
    masterDf = masterDf[masterDf['RISK_FREE_RATE'].notna()]
    
    # Map volatility columns
    masterDf["15DayVol"] = masterDf["QUOTE_DATE"].map(underlying15DVolDict)
    masterDf["30DayVol"] = masterDf["QUOTE_DATE"].map(underlying30DVolDict)
    masterDf["60DayVol"] = masterDf["QUOTE_DATE"].map(underlying60DVolDict)
    masterDf["YZVol"] = masterDf["QUOTE_DATE"].map(underlyingYZVolDict)
    
    # Converting date columns to dates. Requires .date() to become DateTime
    masterDf['EXPIRE_DATE'] = pd.to_datetime(masterDf['EXPIRE_DATE'])
    masterDf['QUOTE_DATE'] = pd.to_datetime(masterDf['QUOTE_DATE'])
    
    # Remove all options past the last date of underlying data
    masterDf = masterDf[~(masterDf['EXPIRE_DATE'] > finalDay)]
    
    # Map option profits
    masterDf['OPTION_PROFIT'] = masterDf.apply(lambda x: getOptionProfit(str(x['EXPIRE_DATE'].date()), x['STRIKE']), axis=1)
    masterDf['BID_ASK_MIDPOINT'] = masterDf.apply(lambda x: (x['C_ASK'] + x['C_BID']) / 2, axis=1)
    return masterDf

def filter_for_moneyness_and_time(df, low_money = None, high_money = None, low_time = None, high_time = None):
    if low_money == None and high_money == None:
        pass
    elif low_money == None:
        df = df[df['STRIKE'] / df['UNDERLYING_LAST'] <= high_money]
    elif high_money == None:
        df = df[df['STRIKE'] / df['UNDERLYING_LAST'] >= low_money]
    else:
        df = df[(df['STRIKE'] / df['UNDERLYING_LAST'] >= low_money) & (df['STRIKE'] / df['UNDERLYING_LAST'] <= high_money)]

    if low_time == None and high_time == None:
        return df
    if low_time == None:
        return df[df['DTE'] <= high_time]
    elif high_time == None:
        return df[df['DTE'] >= low_time]
    else:
        return df[(df['DTE'] >= low_time) & (df['DTE'] <= high_time)]     
    

# For NN, features are: DTE, Underlying last, Delta, gamma, vega, theta, rho, IB

In [3]:
df = getAndProcessOneYearOptionData()
df

C:\Users\David\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,QUOTE_DATE,UNDERLYING_LAST,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,RISK_FREE_RATE,15DayVol,30DayVol,60DayVol,YZVol,OPTION_PROFIT,BID_ASK_MIDPOINT
13,2021-01-04,129.45,2021-01-08,4.00,0.99330,0.00182,0.00267,-0.01615,0.01475,0.70653,...,106.0,23.4,0.181,0.09,0.335185,0.290138,0.345166,0.282196,26.050003,23.475
16,2021-01-04,129.45,2021-01-08,4.00,0.99171,0.00236,0.00355,-0.01585,0.01479,0.61801,...,109.0,20.4,0.158,0.09,0.335185,0.290138,0.345166,0.282196,23.050003,20.475
17,2021-01-04,129.45,2021-01-08,4.00,0.99211,0.00254,0.00301,-0.01615,0.01530,0.58957,...,110.0,19.4,0.150,0.09,0.335185,0.290138,0.345166,0.282196,22.050003,19.480
18,2021-01-04,129.45,2021-01-08,4.00,0.98019,0.00493,0.00688,-0.04014,0.01468,0.65222,...,111.0,18.4,0.143,0.09,0.335185,0.290138,0.345166,0.282196,21.050003,18.520
19,2021-01-04,129.45,2021-01-08,4.00,0.98415,0.00440,0.00543,-0.02860,0.01464,0.58313,...,112.0,17.4,0.135,0.09,0.335185,0.290138,0.345166,0.282196,20.050003,17.495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19210,2021-12-31,177.58,2022-04-14,103.96,0.06265,0.00433,0.11726,-0.01649,0.02991,0.29953,...,230.0,52.4,0.295,0.06,0.293058,0.297586,0.243602,0.311005,0.000000,0.730
19211,2021-12-31,177.58,2022-04-14,103.96,0.05137,0.00364,0.10102,-0.01461,0.02492,0.30623,...,235.0,57.4,0.323,0.06,0.293058,0.297586,0.243602,0.311005,0.000000,0.595
19212,2021-12-31,177.58,2022-04-14,103.96,0.04452,0.00317,0.08905,-0.01318,0.02085,0.31495,...,240.0,62.4,0.351,0.06,0.293058,0.297586,0.243602,0.311005,0.000000,0.515
19213,2021-12-31,177.58,2022-04-14,103.96,0.03869,0.00268,0.08006,-0.01283,0.01835,0.32534,...,245.0,67.4,0.380,0.06,0.293058,0.297586,0.243602,0.311005,0.000000,0.455


In [4]:
filter_for_moneyness_and_time(df, 0.8, 1.2, 30, 90)

,QUOTE_DATE,UNDERLYING_LAST,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,RISK_FREE_RATE,15DayVol,30DayVol,60DayVol,YZVol,OPTION_PROFIT,BID_ASK_MIDPOINT
317,2021-01-04,129.45,2021-02-05,32.00,0.91038,0.00521,0.05748,-0.04362,1.13243,0.50671,...,105.0,24.4,0.189,0.09,0.335185,0.290138,0.345166,0.282196,31.759995,25.020
318,2021-01-04,129.45,2021-02-05,32.00,0.88206,0.00916,0.07433,-0.05288,0.55839,0.46919,...,110.0,19.4,0.150,0.09,0.335185,0.290138,0.345166,0.282196,26.759995,20.425
319,2021-01-04,129.45,2021-02-05,32.00,0.82478,0.01407,0.09978,-0.06695,0.26175,0.44728,...,115.0,14.4,0.112,0.09,0.335185,0.290138,0.345166,0.282196,21.759995,16.125
320,2021-01-04,129.45,2021-02-05,32.00,0.78173,0.01729,0.11437,-0.07418,0.17286,0.42749,...,118.0,11.4,0.088,0.09,0.335185,0.290138,0.345166,0.282196,18.759995,13.625
321,2021-01-04,129.45,2021-02-05,32.00,0.76574,0.01834,0.11905,-0.07611,0.15200,0.42001,...,119.0,10.4,0.081,0.09,0.335185,0.290138,0.345166,0.282196,17.759995,12.795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19164,2021-12-31,177.58,2022-03-18,76.96,0.31659,0.01602,0.29205,-0.05073,0.11061,0.27187,...,190.0,12.4,0.070,0.06,0.293058,0.297586,0.243602,0.311005,0.000000,4.275
19165,2021-12-31,177.58,2022-03-18,76.96,0.24719,0.01423,0.25871,-0.04580,0.08639,0.27164,...,195.0,17.4,0.098,0.06,0.293058,0.297586,0.243602,0.311005,0.000000,3.075
19166,2021-12-31,177.58,2022-03-18,76.96,0.19097,0.01211,0.22244,-0.03926,0.06725,0.27381,...,200.0,22.4,0.126,0.06,0.293058,0.297586,0.243602,0.311005,0.000000,2.225
19167,2021-12-31,177.58,2022-03-18,76.96,0.14702,0.01000,0.18801,-0.03393,0.05166,0.27876,...,205.0,27.4,0.154,0.06,0.293058,0.297586,0.243602,0.311005,0.000000,1.635


In [5]:
def MLP_rmse(low_money, high_money, low_time, high_time, df):
    #start = time.time()
    df = filter_for_moneyness_and_time(df, low_money, high_money, low_time, high_time)
    X = df.iloc[:, [1, 3, 14, 17, 18, 19, 20, 21]].values
    y = df.iloc[:, -1].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)
    
    sc = MinMaxScaler()
    X_train_sc = pd.DataFrame(sc.fit_transform(X_train))
    X_test_sc = pd.DataFrame(sc.transform(X_test))
    
    model = MLPRegressor(activation = 'relu', max_iter = 1000, tol = 0.0001)
    model.fit(X_train_sc, y_train)
    
    y_train_pred = model.predict(X_train_sc)
    y_test_pred = model.predict(X_test_sc)
    print("MLP RMSE on test data for moneyness(" + str(low_money) + ", " + str(high_money) + ") and DTE(" + str(low_time) + ", " + 
          str(high_time) + "): " + str(metrics.mean_squared_error(y_test, y_test_pred) ** 0.5))
    # print("Run in: " + str(time.time() - start) + " sec")
    # return model

In [6]:
MLP_rmse(0.2, 0.8, 0.001, 30, df)
MLP_rmse(0.8, 1.2, 0.001, 30, df)
MLP_rmse(1.2, 1.8, 0.001, 30, df)

MLP_rmse(0.2, 0.8, 30, 90, df)
MLP_rmse(0.8, 1.2, 30, 90, df)
MLP_rmse(1.2, 1.8, 30, 90, df)

MLP_rmse(0.2, 0.8, 90, 300, df)
MLP_rmse(0.8, 1.2, 90, 300, df)
MLP_rmse(1.2, 1.8, 90, 300, df)

MLP RMSE on test data for moneyness(0.2, 0.8) and DTE(0.001, 30): 0.1690928086912491
MLP RMSE on test data for moneyness(0.8, 1.2) and DTE(0.001, 30): 0.27738605251325643
MLP RMSE on test data for moneyness(1.2, 1.8) and DTE(0.001, 30): 0.0540610251326965
MLP RMSE on test data for moneyness(0.2, 0.8) and DTE(30, 90): 0.21028237038169378
MLP RMSE on test data for moneyness(0.8, 1.2) and DTE(30, 90): 0.33366508758855473
MLP RMSE on test data for moneyness(1.2, 1.8) and DTE(30, 90): 0.1059609820937779
MLP RMSE on test data for moneyness(0.2, 0.8) and DTE(90, 300): 0.4025651681546361
MLP RMSE on test data for moneyness(0.8, 1.2) and DTE(90, 300): 0.31624355152335
MLP RMSE on test data for moneyness(1.2, 1.8) and DTE(90, 300): 0.140800052043168


In [7]:
# Code from https://medium.com/swlh/calculating-option-premiums-using-the-black-scholes-model-in-python-e9ed227afbee
def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*T)/(sigma*sqrt(T))
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)
def blackScholes(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))

In [12]:
def BS_rmse(low_money, high_money, low_time, high_time, df, volType):
    df = filter_for_moneyness_and_time(df, low_money, high_money, low_time, high_time)
    n = 0
    squared_error = 0
    for index, row in df.iterrows():
        n += 1
        squared_error += (row['BID_ASK_MIDPOINT'] - blackScholes(row['UNDERLYING_LAST'], row['STRIKE'], row['DTE'] / 252, row['RISK_FREE_RATE'], row[volType])) ** 2
        
    print("BS(" + volType + ") RMSE on test data for moneyness(" + str(low_money) + ", " + str(high_money) + ") and DTE(" + str(low_time) + ", " + 
          str(high_time) + "): " + str((squared_error / n) ** 0.5))

In [13]:
low_money_vals = [0.2, 0.8, 1.2]
high_money_vals = [0.8, 1.2, 1.8]
low_time_vals = [0.001, 30, 90]
high_time_vals = [30, 90, 300]
volTypes = ['15DayVol', '30DayVol', '60DayVol', 'YZVol']

for i in range(len(low_money_vals)):
    low_money = low_money_vals[i]
    high_money = high_money_vals[i]
    for j in range(len(low_time_vals)):
        low_time = low_time_vals[i]
        high_time = high_time_vals[i]
        for k in range(len(volTypes)):
            volType = volTypes[k]
            BS_rmse(low_money, high_money, low_time, high_time, df, volType)

BS(15DayVol) RMSE on test data for moneyness(0.2, 0.8) and DTE(0.001, 30): 0.2840626369711461
BS(30DayVol) RMSE on test data for moneyness(0.2, 0.8) and DTE(0.001, 30): 0.28240373949233605
BS(60DayVol) RMSE on test data for moneyness(0.2, 0.8) and DTE(0.001, 30): 0.2820850513832019
BS(YZVol) RMSE on test data for moneyness(0.2, 0.8) and DTE(0.001, 30): 0.2824440504766086
BS(15DayVol) RMSE on test data for moneyness(0.2, 0.8) and DTE(0.001, 30): 0.2840626369711461
BS(30DayVol) RMSE on test data for moneyness(0.2, 0.8) and DTE(0.001, 30): 0.28240373949233605
BS(60DayVol) RMSE on test data for moneyness(0.2, 0.8) and DTE(0.001, 30): 0.2820850513832019
BS(YZVol) RMSE on test data for moneyness(0.2, 0.8) and DTE(0.001, 30): 0.2824440504766086
BS(15DayVol) RMSE on test data for moneyness(0.2, 0.8) and DTE(0.001, 30): 0.2840626369711461
BS(30DayVol) RMSE on test data for moneyness(0.2, 0.8) and DTE(0.001, 30): 0.28240373949233605
BS(60DayVol) RMSE on test data for moneyness(0.2, 0.8) and DTE(